In [1]:
import os
import opensmile
import pandas as pd
import numpy as np
from natsort import natsorted
import pandas as pd
from pydub import AudioSegment as AS
from ChaFile import *
from difflib import SequenceMatcher as sm


smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

dir_healthy_wav = input("enter the (full) path to your directory of healthy WAV files --> ")
dir_healthy_cha = input("enter the (full) path to your directory of healthy CHA files --> ")

dir_AD_wav = input("enter the (full) path to your directory of AD WAV files --> ")
dir_AD_cha = input("enter the (full) path to your directory of AD CHA files --> ")

dir_MCI_wav = input("enter the (full) path to your directory of MCI WAV files --> ")
dir_MCI_cha = input("enter the (full) path to your directory of MCI CHA files --> ")


In [2]:
def checker(wav, cha):
    files_from_dir = os.listdir(wav)
    files_from_dir_2 = os.listdir(cha)
    if len(files_from_dir) != len(files_from_dir_2):
        quit("not equal")

checker(dir_healthy_wav, dir_healthy_cha)
checker(dir_AD_wav, dir_AD_cha)
checker(dir_MCI_wav, dir_MCI_cha)

AD_cha_files = natsorted(os.listdir(dir_AD_cha))
AD_wav_files = natsorted(os.listdir(dir_AD_wav))

MCI_cha_files = natsorted(os.listdir(dir_MCI_cha))
MCI_wav_files = natsorted(os.listdir(dir_MCI_wav))

Healthy_cha_files = natsorted(os.listdir(dir_healthy_cha))
Healthy_wav_files = natsorted(os.listdir(dir_healthy_wav))

In [3]:
keys_1 = ["command_1", "command_2", "command_3", "command_4", "command_5", "command_6", "command_7", "command_8", "command_9", 
"command_10", "command_11", "command_12", "command_13", "command_14", "command_15", "command_16", "command_17", "command_18", "command_19",
"command_20", "command_21", "command_22", "command_23", "command_24", "command_25", "command_26", "command_27", "command_28", "command_29",
"command_30", "command_31", "command_32", "command_33"]

# keywords = ["weather", "date", "time", "Thanksgiving", ["chocolate", "chip", "cookies"], ["two", "four"], "tablespoons", "symptom", "classical",
# ["volume", "eight"], "jazz", ["volume", "six"], "stop", "laundry", "dog", "reminders", "birthday", 
# ["five", "seconds"], "alarm", ["oranges", "grapes"], ["what", "is", "in", "shopping"], "call", ["hang", "up"], "find", "quit", ["bedroom", "light", "on"],
# "red", "brightness", ["bedroom", "light", "off"], ["open", "kitchen", "camera"], "hide", ["white", "collar"], "pause", "yes"]

COMMANDS = ["Alexa what is the weather outside ? ",
"Alexa what is today's date ? ", 
"Alexa what time is it ? ", 
"Alexa when is Thanksgiving ? ",
"Alexa how do you bake chocolate chip cookies ? ",
"Alexa what is two times four ? ",
"Alexa how many tablespoons in a cup ? ",
"Alexa how do you spell symptom ? ",
"Alexa play classical music . ",
"Alexa volume eight . ", 
"Alexa play jazz . ",
"Alexa volume six . ",
"Alexa stop . ",
"Alexa remind me to start the laundry tomorrow at two p_m . ",
"Alexa remind me to feed the dog at seven p_m everyday . ",
"Alexa tell me my reminders . ",
"Alexa remember my daughter's birthday is June first . ", 
"Alexa set a timer in five seconds . ",
"Alexa set my alarm for seven a_m tomorrow . ",
"Alexa add oranges and grapes to my shopping list . ",
"Alexa what is in my shopping list ? ",
"Alexa call six zero three six six zero two two zero three . ",
"Alexa hang up . ",
"Alexa find my phone . ",
"Alexa quit . ",
"Alexa turn the bedroom light on . ",
"Alexa turn the bedroom light red . ",
"Alexa change brightness to ten . ",
"Alexa turn off the bedroom light . ",
"Alexa open the kitchen camera . ",
"Alexa hide the kitchen camera . ",
"Alexa play white collar on fire tv . ",
"Alexa pause . "]


In [9]:
def unique():
    #Identifies unique words in commands
    keywords = []
    for i in range(len(COMMANDS)):
        word_cloud = COMMANDS[:]
        word_cloud.remove(COMMANDS[i])
        cloud = []
        for com in word_cloud:
            com = com.split(" ")
            for word in com:
                cloud.append(word)
        current = COMMANDS[i].split(" ")
        sublist = []
        for uni in current:
            if uni in cloud:
            #count += 1
                pass
            else:
                sublist.append(uni)
        keywords.append(sublist)
    print(keywords)
    print(len(COMMANDS))
    print(len(keywords))
    return keywords

keywords = unique()


[['weather', 'outside'], ["today's", 'date'], ['time', 'it'], ['when', 'Thanksgiving'], ['bake', 'chocolate', 'chip', 'cookies'], ['times', 'four'], ['many', 'tablespoons', 'cup'], ['spell', 'symptom'], ['classical', 'music'], ['eight'], ['jazz'], [], ['stop'], ['start', 'laundry'], ['feed', 'dog', 'everyday'], ['tell', 'reminders'], ['remember', "daughter's", 'birthday', 'June', 'first'], ['timer', 'five', 'seconds'], ['alarm', 'for', 'a_m'], ['add', 'oranges', 'and', 'grapes'], [], ['call', 'zero', 'three', 'zero', 'zero', 'three'], ['hang', 'up'], ['find', 'phone'], ['quit'], [], ['red'], ['change', 'brightness', 'ten'], ['off'], ['open'], ['hide'], ['white', 'collar', 'fire', 'tv'], ['pause']]
33
33


In [5]:
keywords[11] = ["Alexa", "volume", "six"]
keywords[21] = ["call"]
keywords[20] = ["Alexa", "what", "is", "in", "my", "shopping", "list"]
keywords[25] = ["Alexa", "turn", "the", "bedroom", "light", "on"]
#These are the non-unique commands containing no unique words
#Maybe try 2-keywords? Or how to deal with (Volume six / Volume eight) <---
dz = dict(zip(keys_1, keywords))

In [51]:
def distance(str1, str2):
    sql = sm(None, str1, str2)
    return sql.ratio()


def split_audio(timestamp, wav_dir, wav_file, cha_dir, participant, command_keys, custom_folder):
    temp = ("C:\\Users\\EliKurtz\\VSCode_files\\NLP\\Kurtz-NLP\\Audio gEMAPS\\temp")
    # full = ("C:\\Users\\EliKurtz\\VSCode_files\\NLP\\Kurtz-NLP\\Audio gEMAPS")
    full = custom_folder
    print(timestamp)
    os.chdir(wav_dir)
    part = AS.from_file(wav_file)
    t = any(isinstance(i, list) for i in timestamp)
    if not t:
        if len(timestamp) >= 1:
            clip = part[timestamp[0]:timestamp[1]]
            name = (participant + "_" + str(command_keys) + ".wav")
            os.chdir(full)
            clip.export(name, format="wav")
    else:
        count = 0
        for time in timestamp:
            print(timestamp)
            clip = part[time[0]:time[1]] 
            name = (participant + "_" + str(command_keys) + str(count) + ".wav")
            os.chdir(temp)
            clip.export(name, format="wav")
            count += 1
            print(os.listdir(temp))
        tempfiles = os.listdir(temp)
        combined_wav = AS.empty()
        for file in tempfiles:
            sound = AS.from_file(file, format="wav")
            combined_wav += sound
            os.remove(file)
        os.chdir(full)
        combined_wav.export(name, format="wav")
    os.chdir(cha_dir)


def categorize(cha_dir, wav_dir, cha_file, wav_file):
    os.chdir(cha_dir)
    cha = ChaFile(cha_file)
    lines = cha.getLines()
    data_set = []
    participant = cha_file
    unmatched_count = 1
    ur_commands_times = []
    um_command_times = []
    os.mkdir("C:\\Users\\EliKurtz\\VSCode_files\\NLP\\Kurtz-NLP\\Audio gEMAPS\\" + str(participant))
    custom_folder = "C:\\Users\\EliKurtz\\VSCode_files\\NLP\\Kurtz-NLP\\Audio gEMAPS\\" + str(participant)
    print(custom_folder)
    for data in lines:
        action_taken = 0
        utterance = data["xol"]
        timestamp = data["bullet"]
        utterance = utterance.split(" ")
        #print(utterance)
        dist = []
        for com in COMMANDS:
            dist.append(distance(com, data["xol"]))
        closest_neighbor = COMMANDS[dist.index(max(dist))]
        percent_match = (max(dist))
        #Identifies closest neighbor
        if utterance[0][0] == "[":
                #print("FLAG")
                keys = "UNRECOGNIZED COMMAND"
                closest_neighbor = "N/A"
                percent_match = "N/A"
                data_set.append([participant, keys, 0, utterance, timestamp, percent_match, closest_neighbor])
                ur_commands_times.append(timestamp)
        else:        
            for word in dz.values():
                if type(word) is list:
                    key_list = []
                    c = all(x in utterance for x in word)
                    if c:
                        action_taken += 1
                        keys = [k for k, v in dz.items() if v == word]
                        key_list.append(keys)
                        # print("matched", word, keys, timestamp)
                        data_set.append([participant, key_list, word, utterance, timestamp, percent_match, closest_neighbor])
                        split_audio(timestamp, wav_dir, wav_file, cha_dir, participant, key_list, custom_folder)
                        print("calling from 1")
                if word in utterance:
                    action_taken = 1
                    keys = [k for k, v in dz.items() if v == word]
                    #print("matched", word, keys, timestamp)
                    data_set.append([participant, keys, word, utterance, timestamp, percent_match, closest_neighbor])
                    split_audio(timestamp, wav_dir, wav_file, cha_dir, participant, key_list, custom_folder)
                    print("calling from 2")
            if action_taken == 0:
                keys = ["UNMATCHED COMMAND #" + str(unmatched_count)] 
                #print("FAILED TO MATCH")
                data_set.append([participant, keys, 0, utterance, timestamp, percent_match, closest_neighbor])
                unmatched_count += 1
                um_command_times.append(timestamp)
    if len(um_command_times) > 1:
        print(len(um_command_times))
        print(um_command_times)
        split_audio(um_command_times, wav_dir, wav_file, cha_dir, participant, "UNMATCHED", custom_folder)
        print("calling from 3")
    if len(um_command_times) > 1:
        split_audio(ur_commands_times, wav_dir, wav_file, cha_dir, participant, "UNRECOGNIZED", custom_folder)
    frame = pd.DataFrame(data_set)
    print("calling from 4")
    frame.columns = ["Participant", "Matched Command", "Unique Key-Words", 
    "Utterance", "Timestamp", "% Command-Utterance Match", "Closest Neighbor"]
    print(frame)
    os.chdir("..")
    # with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer:
    #     frame.to_excel(writer, sheet_name=participant)    

In [52]:
for i in range(len(AD_cha_files)):
    categorize(dir_AD_cha, dir_AD_wav, AD_cha_files[i], AD_wav_files[i])

for i in range(len(MCI_cha_files)):
    categorize(dir_MCI_cha, dir_MCI_wav, MCI_cha_files[i], MCI_wav_files[i])

for i in range(len(Healthy_cha_files)):
    categorize(dir_healthy_cha, dir_healthy_wav, Healthy_cha_files[i], Healthy_wav_files[i])


C:\Users\EliKurtz\VSCode_files\NLP\Kurtz-NLP\Audio gEMAPS\012.cha
[0, 3250]
calling from 1
[3250, 5750]
calling from 1
[5750, 8490]
calling from 1
[8490, 11680]
calling from 1
[11680, 15530]
calling from 1
[15530, 18210]
calling from 1
[18210, 21580]
calling from 1
[21580, 24590]
calling from 1
[24590, 27780]
calling from 1
[27780, 30370]
calling from 1
[30370, 33390]
calling from 1
[33390, 37100]
calling from 1
[37100, 39720]
calling from 1
[39720, 41800]
calling from 1
[41800, 47600]
calling from 1
[47600, 53190]
calling from 1
[53190, 56380]
calling from 1
[56380, 60260]
calling from 1
[73580, 75480]
calling from 1
[75480, 79900]
calling from 1
[79900, 84980]
calling from 1
[87870, 91360]
calling from 1
[91360, 96530]
calling from 1
[96530, 102360]
calling from 1
[105580, 108410]
calling from 1
[108410, 111870]
calling from 1
[111870, 114760]
calling from 1
[114760, 117650]
calling from 1
[117650, 120720]
calling from 1
[120720, 124600]
calling from 1
[124600, 130080]
calling from 1